# Regresión

Autores:
Ivan Toribio
Héctor Sancho
Jordi Hamberg

# Enunciado

En esta práctica, se os proporcionará un conjunto de datos con características de automóviles de segunda mano en distintos países. El objetivo es preprocesar los datos y construir un modelo capaz de predecir los precios de coches que nunca haya visto.

Además, se llevará a cabo una **competición** en kaggle, podéis inscribiros en este siguiente [enlace](https://www.kaggle.com/t/95194d38b1f9447fa9e3b51b8e9d575f)




La estructura del proyecto debe ser la siguiente:

- Carga y limpieza de los datos
- Visualización
- Preprocesamiento
- Pruebas de modelos (razonar sobre los resultados)
- Conclusiones
- Creación de submission



Se valorarán:

- Número y variedad de experimentos con sus conclusiones.
- Calidad del preprocesamiento (y su razonamiento).
- Capacidad de generalización del modelo final.
- Trabajo en equipo (en grupos de más de 1 integrante, claro).
- Creatividad en la solución.
- Claridad del .ipynb presentado en Moodle

# Carga y limpieza de los datos


Aunque se permite la carga de los datos de forma local, se recomienda emplear las siguientes URLs de gist por comodidad:

In [127]:
import numpy as np

In [128]:
train_url = "https://gist.githubusercontent.com/w-dan/9c2f9a30d0517c58a08d5a7ef42ef2dd/raw/ee85f61926c9aeae5863369a13bb67e182a702ba/training.csv"
test_url = "https://gist.githubusercontent.com/w-dan/9c2f9a30d0517c58a08d5a7ef42ef2dd/raw/ee85f61926c9aeae5863369a13bb67e182a702ba/test.csv"

In [129]:
import pandas as pd

train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

In [130]:
import matplotlib.pyplot as plt

# Visualización

In [131]:
train_df

,id,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,1,Chevrolet Spark 1.0 LT,2011,130000,68519,Petrol,Dealer,Manual,First Owner,16.2 kmpl,995 CC,62.1 bhp,90.3Nm@ 4200rpm,5.0
1,2,Hyundai Verna 1.6 SX CRDi (O),2017,1050000,70000,Diesel,Dealer,Manual,First Owner,22.32 kmpl,1582 CC,126.32 bhp,259.8Nm@ 1900-2750rpm,5.0
2,3,Mahindra Scorpio S10 7 Seater,2016,1100000,60000,Diesel,Individual,Manual,First Owner,15.4 kmpl,2179 CC,120 bhp,280Nm@ 1800-2800rpm,7.0
3,4,Hyundai EON 1.0 Kappa Magna Plus Optional,2018,350000,10000,Petrol,Individual,Manual,Second Owner,20.3 kmpl,998 CC,68.05 bhp,94.14Nm@ 3500rpm,5.0
4,5,Maruti Swift Dzire VDI,2014,490000,50000,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6904,6905,Hyundai i20 2015-2017 Asta Option 1.2,2016,650000,29000,Petrol,Individual,Manual,First Owner,18.6 kmpl,1197 CC,81.83 bhp,114.7Nm@ 4000rpm,5.0
6905,6906,Hyundai EON Sportz,2017,300000,10000,Petrol,Individual,Manual,First Owner,21.1 kmpl,814 CC,55.2 bhp,74.5Nm@ 4000rpm,5.0
6906,6907,Maruti Swift VDI Optional,2017,693000,58559,Diesel,Dealer,Manual,First Owner,25.2 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
6907,6908,Maruti Baleno Alpha 1.2,2017,580000,20000,Petrol,Individual,Manual,First Owner,21.4 kmpl,1197 CC,83.1 bhp,115Nm@ 4000rpm,5.0


# Preprocesamiento

In [132]:
processed_df = pd.DataFrame()

In [133]:
def crear_pol(columna : pd.Series, column_name : str):

    DEGREE = 1

    if DEGREE == 1:
        return pd.DataFrame({column_name + "_1": columna})

    columna = columna.replace(np.nan, 0)

    matriz_valores = columna.values.reshape(-1, 1)

    from sklearn.preprocessing import PolynomialFeatures
    pol = PolynomialFeatures(DEGREE)

    matriz = pol.fit_transform(matriz_valores)

    df_temp = pd.DataFrame()

    for i, columna in enumerate(matriz.transpose()):
        df_temp[f"{column_name}_{i}"] = columna

    df_temp = df_temp.replace(0, np.nan)

    return df_temp

In [134]:
#nombre
processed_df["name"] = train_df["name"]

In [135]:
#precio
processed_df["selling_price"] = train_df["selling_price"]

In [136]:
#year
processed_df = pd.concat([processed_df, crear_pol(train_df["year"], "year")], axis=1)

In [137]:
#km_driven
processed_df = pd.concat([processed_df, crear_pol(train_df["km_driven"], "km_driven")], axis=1)

In [138]:
def splitter(x):
    if type(x) == str:
        return float(x.split(" ")[0])
    return x

In [139]:
import re

def splitter_re(x):
    if type(x) == str:
        try:
            return re.findall(r"[0-9.]+", x)[0]
        except Exception:
            return np.nan
    else:
        return x

In [140]:
#mileage

serie = train_df["mileage"]
serie = serie.apply(splitter)

processed_df = pd.concat([processed_df, crear_pol(serie, "mileage")], axis=1)



In [141]:
#engine


serie = train_df["engine"]
serie = serie.apply(splitter)

processed_df = pd.concat([processed_df, crear_pol(serie, "engine")], axis=1)

In [142]:
#max_power

serie = train_df["max_power"]
serie = serie.apply(splitter_re)

processed_df = pd.concat([processed_df, crear_pol(serie, "max_power")], axis=1)

In [143]:
#seats

serie = train_df["seats"]

processed_df = pd.concat([processed_df, crear_pol(serie, "seats")], axis=1)

In [144]:
categoricas = ["fuel", "seller_type", "transmission", "owner"]

diccionarios = {}

In [145]:
df_aux = pd.DataFrame({

    "selling_price":train_df["selling_price"],
    "brand": train_df["name"].map(lambda x: x.split()[0])

})

pesos = df_aux.groupby("brand").mean().reset_index()
pesos = pesos.rename(columns={"selling_price": "peso"})

df_aux.drop(columns=["selling_price"], inplace=True)

diccionario = {categorica : peso for categorica, peso in pesos.values}

diccionarios["brand"] = diccionario

df_aux = df_aux.map(lambda x: diccionario[x])

processed_df = pd.concat([processed_df, crear_pol(df_aux["brand"], "brand")], axis=1)

In [146]:
for categorica in categoricas:

    df_aux = train_df.loc[:,[categorica, "selling_price"]]

    pesos = df_aux.groupby(categorica).mean().reset_index()
    pesos = pesos.rename(columns={"selling_price": "peso"})

    df_aux.drop(columns=["selling_price"], inplace=True)

    diccionario = {categorica : peso for categorica, peso in pesos.values}

    diccionarios[categorica] = diccionario

    df_aux = df_aux.map(lambda x: diccionario[x])

    processed_df = pd.concat([processed_df, crear_pol(df_aux[categorica], categorica)], axis=1)

# Pruebas de modelos

In [147]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(processed_df, test_size=0.2, random_state=43)


X_train = train.drop(["name", "selling_price"], axis=1)
y_train = train["selling_price"]

X_test = test.drop(["name", "selling_price"], axis=1)
y_test = test["selling_price"]

from sklearn.ensemble import HistGradientBoostingRegressor
reg = HistGradientBoostingRegressor()

reg.fit(X_train, y_train)



LinearRegression()

In [148]:
#metricas

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import explained_variance_score as evs
from sklearn.metrics import mean_squared_error as mse

print(mse(y_test, reg.predict(X_test)))
print (mae(y_test, reg.predict(X_test)))
print (mae(y_test, reg.predict(X_test))*0.011)

print (evs(y_test, reg.predict(X_test)))

313923686020.7564
174057.11533041613
1914.6282686345774
0.42045517502626606


In [149]:
df_resultados = test.loc[:, ['name', 'selling_price', 'year_1', 'km_driven_1', 'mileage_1',
       'engine_1', 'max_power_1', 'seats_1', 'brand_1', 'fuel_1',
       'seller_type_1', 'transmission_1', 'owner_1']]

df_resultados["precio"] = df_resultados["selling_price"] * 0.011
df_resultados["prediccion"] = reg.predict(X_test) * 0.011
df_resultados["diferencia"] = (df_resultados["prediccion"] - df_resultados["precio"])
df_resultados["rel_diferencia"] = (df_resultados["diferencia"] / df_resultados["precio"]) * 100

df_resultados

,name,selling_price,year_1,km_driven_1,mileage_1,engine_1,max_power_1,seats_1,brand_1,fuel_1,seller_type_1,transmission_1,owner_1,precio,prediccion,diferencia,rel_diferencia
1108,Hyundai i20 Sportz Option 1.2,400000,2014.0,147000.0,18.60,1197.0,81.83,5.0,454804.856073,459605.076059,4.920603e+05,4.530914e+05,385728.259053,4400.0,2465.036316,-1934.963684,-43.976447
1478,Maruti Vitara Brezza ZDi Plus,850000,2016.0,100000.0,24.30,1248.0,88.50,5.0,405203.436681,778718.659924,4.920603e+05,4.530914e+05,771708.327317,9350.0,5567.810611,-3782.189389,-40.451223
6757,Renault Lodgy 85PS RxE,600000,2015.0,125000.0,21.04,1461.0,83.80,8.0,462474.454082,778718.659924,4.920603e+05,4.530914e+05,771708.327317,6600.0,5289.299676,-1310.700324,-19.859096
53,Honda City i-VTEC CVT V,1085000,2018.0,1000.0,18.00,1497.0,117.60,5.0,601355.501259,459605.076059,4.920603e+05,1.833223e+06,771708.327317,11935.0,10574.144042,-1360.855958,-11.402228
3714,Honda Amaze S i-Dtech,520000,2014.0,48000.0,25.80,1498.0,98.60,5.0,601355.501259,778718.659924,4.920603e+05,4.530914e+05,771708.327317,5720.0,6625.408445,905.408445,15.828819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6442,Hyundai Creta 1.6 CRDi AT S Plus,1143000,2017.0,51000.0,17.01,1582.0,126.20,5.0,454804.856073,778718.659924,4.920603e+05,1.833223e+06,771708.327317,12573.0,10566.008523,-2006.991477,-15.962710
4875,Hyundai Grand i10 AT Asta,465000,2015.0,63063.0,18.90,1197.0,82.00,5.0,454804.856073,459605.076059,1.415780e+06,1.833223e+06,771708.327317,5115.0,6101.990935,986.990935,19.296010
1285,Hyundai i20 Asta 1.4 CRDi (Diesel),320000,2010.0,135000.0,23.00,1396.0,90.00,5.0,454804.856073,778718.659924,4.920603e+05,4.530914e+05,385728.259053,3520.0,2495.511013,-1024.488987,-29.104801
1411,Nissan Sunny Diesel XV,300000,2012.0,90000.0,21.64,1461.0,84.80,5.0,458236.055556,778718.659924,4.920603e+05,4.530914e+05,771708.327317,3300.0,4270.955298,970.955298,29.422888


In [150]:
processed_df

,name,selling_price,year_0,year_1,year_2,year_3,km_driven_0,km_driven_1,km_driven_2,km_driven_3,...,seller_type_2,seller_type_3,transmission_0,transmission_1,transmission_2,transmission_3,owner_0,owner_1,owner_2,owner_3
0,Chevrolet Spark 1.0 LT,130000,1.0,2011.0,4044121.0,8.132727e+09,1.0,68519.0,4.694853e+09,3.216867e+14,...,2.004432e+12,2.837834e+18,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,771708.327317,5.955337e+11,4.595783e+17
1,Hyundai Verna 1.6 SX CRDi (O),1050000,1.0,2017.0,4068289.0,8.205739e+09,1.0,70000.0,4.900000e+09,3.430000e+14,...,2.004432e+12,2.837834e+18,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,771708.327317,5.955337e+11,4.595783e+17
2,Mahindra Scorpio S10 7 Seater,1100000,1.0,2016.0,4064256.0,8.193540e+09,1.0,60000.0,3.600000e+09,2.160000e+14,...,2.421233e+11,1.191393e+17,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,771708.327317,5.955337e+11,4.595783e+17
3,Hyundai EON 1.0 Kappa Magna Plus Optional,350000,1.0,2018.0,4072324.0,8.217950e+09,1.0,10000.0,1.000000e+08,1.000000e+12,...,2.421233e+11,1.191393e+17,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,385728.259053,1.487863e+11,5.739108e+16
4,Maruti Swift Dzire VDI,490000,1.0,2014.0,4056196.0,8.169179e+09,1.0,50000.0,2.500000e+09,1.250000e+14,...,2.421233e+11,1.191393e+17,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,771708.327317,5.955337e+11,4.595783e+17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6904,Hyundai i20 2015-2017 Asta Option 1.2,650000,1.0,2016.0,4064256.0,8.193540e+09,1.0,29000.0,8.410000e+08,2.438900e+13,...,2.421233e+11,1.191393e+17,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,771708.327317,5.955337e+11,4.595783e+17
6905,Hyundai EON Sportz,300000,1.0,2017.0,4068289.0,8.205739e+09,1.0,10000.0,1.000000e+08,1.000000e+12,...,2.421233e+11,1.191393e+17,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,771708.327317,5.955337e+11,4.595783e+17
6906,Maruti Swift VDI Optional,693000,1.0,2017.0,4068289.0,8.205739e+09,1.0,58559.0,3.429156e+09,2.008080e+14,...,2.004432e+12,2.837834e+18,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,771708.327317,5.955337e+11,4.595783e+17
6907,Maruti Baleno Alpha 1.2,580000,1.0,2017.0,4068289.0,8.205739e+09,1.0,20000.0,4.000000e+08,8.000000e+12,...,2.421233e+11,1.191393e+17,1.0,453091.397115,2.052918e+11,9.301595e+16,1.0,771708.327317,5.955337e+11,4.595783e+17


# Conclusiones

# Creación de *submission* para Kaggle

In [151]:
#procesado dataset

processed_sub_df = pd.DataFrame()

#nombre
processed_sub_df["name"] = test_df["name"]

#year
processed_sub_df = pd.concat([processed_sub_df, crear_pol(test_df["year"], "year")], axis=1)

#km_driven
processed_sub_df = pd.concat([processed_sub_df, crear_pol(test_df["km_driven"], "km_driven")], axis=1)

#mileage

serie = test_df["mileage"]
serie = serie.apply(splitter)

processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, "mileage")], axis=1)

#engine


serie = test_df["engine"]
serie = serie.apply(splitter)

processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, "engine")], axis=1)

#max_power

serie = test_df["max_power"]
serie = serie.apply(splitter_re)

processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, "max_power")], axis=1)

#seats

serie = test_df["seats"]

processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, "seats")], axis=1)

#Categoricas

serie_marcas = test_df["name"].apply(lambda x: x.split(" ")[0])
serie_marcas = serie_marcas.map(lambda x: diccionarios["brand"][x])
processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie_marcas, "brand")], axis=1)



categoricas = ["fuel", "seller_type", "transmission", "owner"]

for categorica in categoricas:
    serie = test_df[categorica].map(lambda x: diccionarios[categorica][x])
    processed_sub_df = pd.concat([processed_sub_df, crear_pol(serie, categorica)], axis=1)


print(processed_sub_df.columns)
print(processed_df.columns)


    



Index(['name', 'year_0', 'year_1', 'year_2', 'year_3', 'km_driven_0',
       'km_driven_1', 'km_driven_2', 'km_driven_3', 'mileage_0', 'mileage_1',
       'mileage_2', 'mileage_3', 'engine_0', 'engine_1', 'engine_2',
       'engine_3', 'max_power_0', 'max_power_1', 'max_power_2', 'max_power_3',
       'seats_0', 'seats_1', 'seats_2', 'seats_3', 'brand_0', 'brand_1',
       'brand_2', 'brand_3', 'fuel_0', 'fuel_1', 'fuel_2', 'fuel_3',
       'seller_type_0', 'seller_type_1', 'seller_type_2', 'seller_type_3',
       'transmission_0', 'transmission_1', 'transmission_2', 'transmission_3',
       'owner_0', 'owner_1', 'owner_2', 'owner_3'],
      dtype='object')
Index(['name', 'selling_price', 'year_0', 'year_1', 'year_2', 'year_3',
       'km_driven_0', 'km_driven_1', 'km_driven_2', 'km_driven_3', 'mileage_0',
       'mileage_1', 'mileage_2', 'mileage_3', 'engine_0', 'engine_1',
       'engine_2', 'engine_3', 'max_power_0', 'max_power_1', 'max_power_2',
       'max_power_3', 'seats_0', 'se

In [152]:
X_sub = processed_sub_df.drop("name", axis=1)
prediccion = reg.predict(X_sub)


prediccion

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from csv import writer

with open("resultados.csv", "w") as file:
    escritor = writer(file, lineterminator="\n")

    escritor.writerow(["id", "TARGET"])

    for i, dato in enumerate(prediccion):
        escritor.writerow([i + 1,dato])